In [11]:
import pandas as pd
from datetime import datetime
from neo4j import GraphDatabase, RoutingControl

# Extract & Transform

In [94]:
df_org = pd.read_csv("../data/car_prices.csv")
df_org = df_org.dropna(subset=["saledate"])

def xfm(r):
    saledate = float("nan")
    try:
        saledate = datetime.strptime(r["saledate"][:-6],"%a %b %d %Y %H:%M:%S GMT%z")
    except ValueError:
        pass
    r["saledate"] = saledate
    return r
df_org = df_org.apply(xfm, axis=1)
df_org = df_org.dropna()


In [89]:
df_org.head(3)

,year,make,model,trim,body,transmission,vin,state,condition,odometer,color,interior,seller,mmr,sellingprice,saledate
0,2015,Kia,Sorento,LX,SUV,automatic,5xyktca69fg566472,ca,5.0,16639.0,white,black,kia motors america inc,20500.0,21500.0,2014-12-16 12:30:00-08:00
1,2015,Kia,Sorento,LX,SUV,automatic,5xyktca69fg561319,ca,5.0,9393.0,white,beige,kia motors america inc,20800.0,21500.0,2014-12-16 12:30:00-08:00
2,2014,BMW,3 Series,328i SULEV,Sedan,automatic,wba3c1c51ek116351,ca,45.0,1331.0,gray,black,financial services remarketing (lease),31900.0,30000.0,2015-01-15 04:30:00-08:00


# Load

In [83]:
URI = "neo4j://localhost:7687"
AUTH = ("neo4j", "neo4j!!!!")

In [110]:
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    for tup in df_org.iterrows():
        r = tup[1]
        model_year = r["year"]
        make = r["make"]
        model = r["model"]
        trim = r["trim"]
        body = r["body"]
        transmission = r["transmission"]
        vin = r["vin"]
        odometer = r["odometer"]
        extcolor = r["color"]
        intcolor = r["interior"]
        seller = r["seller"]
        mmr = r["mmr"]
        sellingprice = r["sellingprice"]
        saledate = r["saledate"]
        
        driver.execute_query(
            "MERGE (v:Vehicle {vin: $vin, trim: $trim, odometer:$odometer, exterior_color:$extcolor, interior_color:$intcolor, mmr:$mmr, sellingprice:$sellingprice, saledate:$saledate}) "
            "MERGE (mk:Maker {maker: $make}) "
            "MERGE (mdl:Model {model: $model}) "
            "MERGE (bd:Body {body: $body}) "
            "MERGE (tx:Transmission {transmission: $transmission})"
            "MERGE (s:Seller {seller: $seller}) "
            
            "MERGE (mk)-[:MADE]->(v)"
            "MERGE (v)-[:IS_A]->(mdl)"
            "MERGE (mdl)-[:IS_A]->(bd)"
            "MERGE (v)-[:HAS_TRANSMISSION]->(tx)"
            "MERGE (s)-[:SOLD]->(v)",
            
            model_year=model_year,
            make=make,
            model=model,
            trim=trim,
            body=body,
            transmission=transmission,
            vin=vin,
            odometer=odometer,
            extcolor=extcolor,
            intcolor=intcolor,
            seller=seller,
            mmr=mmr,
            sellingprice=sellingprice,
            saledate=saledate,            
            database_="neo4j",
        )

KeyboardInterrupt: 